In [134]:
import pandas as pd

# Завантаження даних з файлу Excel
df = pd.read_excel('lab6.xlsx')

df.head()

,N,file name,image size,glasses color,line width
0,1,emma-watson2.jpg,300x300,red,2
1,2,emma-watson.jpg,400x400,blue,3
2,3,draco.jpg,700x700,cian,4
3,4,longbottom.jpg,600x600,magenta,5
4,5,ron_wesley.jpg,500x500,yellow,6


In [26]:
import cv2
import pandas as pd
import numpy as np

# Визначаємо номер варіанту
X = 'D'  # Перша велика літера вашого імені
N = ord(X) % 5 + 1
print("Номер варіанту:", N)

# Завантаження даних з файлу Excel
df = pd.read_excel('lab6.xlsx')

# Отримання даних для варіанту
data_variant = df.loc[df['N'] == N]
print("Дані для варіанту:")
print(data_variant)

# Функція для обробки зображення та додавання окулярів
def process_image_from_dataframe(data_row, output_path):
    """
    Обробляє зображення з додаванням окулярів, використовуючи дані з рядка датафрейму.
    
    Аргументи:
        data_row (pd.Series): Рядок із датафрейму, що містить шлях до файлу, параметри кольору та товщини.
        output_path (str): Шлях для збереження обробленого зображення.
    """
    # Завантаження Haar Cascade для пошуку обличчя та очей
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
    
    # Отримання параметрів із датафрейму
    image_path = data_row['file name']  # Шлях до файлу
    color_name = data_row['glasses color']  # Колір окулярів (сховище у вигляді тексту)
    thickness = data_row['line width']  # Товщина лінії
    
    # Визначення кольору окулярів на основі назви
    color_dict = {
        'red': (0, 0, 255),
        'blue': (255, 0, 0),
        'cian': (255, 255, 0),
        'magenta': (255, 0, 255),
        'yellow': (0, 255, 255)
    }
    color = color_dict.get(color_name, (0, 0, 0))  # За замовчуванням чорний, якщо колір не знайдено

    # Завантаження зображення
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Знаходження обличчя
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100))
    
    for (x, y, w, h) in faces:
        # Вирізаємо обличчя та шукаємо очі
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)
        
        # Малюємо окуляри у вигляді -0-0-
        if len(eyes) >= 2:  # Упевнимося, що є два ока
            eyes = sorted(eyes, key=lambda e: e[0])  # Сортуємо очі за X
            left_eye, right_eye = eyes[:2]
            
            # Координати очей
            lx, ly = x + left_eye[0] + left_eye[2] // 2, y + left_eye[1] + left_eye[3] // 2
            rx, ry = x + right_eye[0] + right_eye[2] // 2, y + right_eye[1] + right_eye[3] // 2
            
            # Малюємо ліву лінзу
            radius = max(left_eye[2], left_eye[3]) // 2
            cv2.circle(img, (lx, ly), radius, color, thickness)
            
            # Малюємо праву лінзу
            radius = max(right_eye[2], right_eye[3]) // 2
            cv2.circle(img, (rx, ry), radius, color, thickness)
            
            # Малюємо перенісся
            cv2.line(img, (lx + radius, ly), (rx - radius, ry), color, thickness)
            
            # Малюємо дужки окулярів
            cv2.line(img, (lx - radius - 20, ly), (lx - radius, ly), color, thickness)
            cv2.line(img, (rx + radius, ry), (rx + radius + 20, ry), color, thickness)
    
    # Зменшення розміру зображення
    face_center = img[y:y+h, x:x+w]  # Вирізаємо обличчя
    reduced_img = cv2.resize(face_center, (600, 600))  # Зменшуємо до розміру 600x600

    # Збереження обробленого зображення
    cv2.imwrite(output_path, reduced_img)
    print(f"Зображення збережено за шляхом: {output_path}")


# Виклик функції для обробки зображення для вашого варіанту
output_path = 'output_variant.jpg'
process_image_from_dataframe(data_variant.iloc[0], output_path)

Номер варіанту: 4
Дані для варіанту:
   N       file name image size glasses color  line width
3  4  longbottom.jpg    600x600       magenta           5
Зображення збережено за шляхом: output_variant.jpg
